# 1P

In [1]:
import pickle
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import  classification_report, confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedShuffleSplit

from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier

#試取資料
file = open("./log/P1(1).pickle", "rb")
data = pickle.load(file)
file.close()
type(data['ml_1P'])

dict

In [2]:
game_info = data["ml_1P"]["scene_info"]
game_command = data["ml_1P"]["command"]
print(game_info)
print(game_command)

[{'frame': 0, 'status': 'GAME_ALIVE', 'ball': (98, 80), 'ball_speed': (0, 0), 'platform_1P': (80, 420), 'platform_2P': (80, 50), 'blocker': (40, 240)}, {'frame': 1, 'status': 'GAME_ALIVE', 'ball': (98, 80), 'ball_speed': (0, 0), 'platform_1P': (80, 420), 'platform_2P': (80, 50), 'blocker': (35, 240)}, {'frame': 2, 'status': 'GAME_ALIVE', 'ball': (98, 80), 'ball_speed': (0, 0), 'platform_1P': (80, 420), 'platform_2P': (80, 50), 'blocker': (30, 240)}, {'frame': 3, 'status': 'GAME_ALIVE', 'ball': (98, 80), 'ball_speed': (0, 0), 'platform_1P': (80, 420), 'platform_2P': (80, 50), 'blocker': (25, 240)}, {'frame': 4, 'status': 'GAME_ALIVE', 'ball': (98, 80), 'ball_speed': (0, 0), 'platform_1P': (80, 420), 'platform_2P': (80, 50), 'blocker': (20, 240)}, {'frame': 5, 'status': 'GAME_ALIVE', 'ball': (98, 80), 'ball_speed': (0, 0), 'platform_1P': (80, 420), 'platform_2P': (80, 50), 'blocker': (15, 240)}, {'frame': 6, 'status': 'GAME_ALIVE', 'ball': (98, 80), 'ball_speed': (0, 0), 'platform_1P': (

In [3]:
for i in range(38, 88):  #71開始18 17個 2-88
    path = "./log/P1(" + str(i) + ").pickle"
    file = open(path, "rb")
    data = pickle.load(file)
    game_info = game_info + data['ml_1P']['scene_info']
    game_command = game_command + data['ml_1P']['command']
    file.close()  
for i in range(45, 95):  #78開始18   17個  9-95
    path = "./log/P2(" + str(i) + ").pickle"
    file = open(path, "rb")
    data = pickle.load(file)
    game_info = game_info + data['ml_1P']['scene_info']
    game_command = game_command + data['ml_1P']['command']
    file.close() 
print(len(game_info))
print(len(game_command))

150694
150694


In [4]:
g = game_info[1]

feature = np.array([g["ball"][0], g["ball"][1], g["ball_speed"][0], g["ball_speed"][1], g["platform_1P"][0]+15,80])
print(feature)
print(game_command[1])
if game_command[1] == "NONE":
	game_command[1] = 0
elif game_command[1] == "MOVE_LEFT":
	game_command[1] = 1
else:
	game_command[1] = 2

[98 80  0  0 95 80]
None


In [5]:
for i in range(2, len(game_info) - 1):
    g = game_info[i]
    g_last = game_info[i-1]
    ball_x=g["ball"][0]
    ball_y=g["ball"][1]
    ball_vx=g['ball'][0] - g_last['ball'][0]
    ball_vy=g['ball'][1] - g_last['ball'][1]
    des_x = 80
    
    if ball_vy > 0:                                             #ball go down
            if ball_vx < 0:                                         #ball move left
                des_x = ball_x - (420 - ball_y)
            else:                                                       #ball move right
                des_x = ball_x + (420 - ball_y)
    else:                                                           #ball move up  
            if ball_vx < 0:
                des_x = ball_x - (680 - (420 - ball_y))
            else:
                des_x = ball_x + (680 - (420 - ball_y))
    while des_x > 200 or des_x < 0:
        if des_x > 200:
                des_x = 200 - (des_x - 200)
        else: 
                des_x = -des_x
    feature = np.vstack((feature, [g['ball'][0], g['ball'][1],ball_vx, ball_vy,g['platform_1P'][0]+15,des_x]))
    if game_command[i] == "NONE": game_command[i] = 0
    elif game_command[i] == "MOVE_LEFT": game_command[i] = 1
    else: game_command[i] = 2 
        
answer = np.array(game_command[1:-1])
print(feature)
print(feature.shape)
print(answer)
print(answer.shape)

[[ 98  80   0   0  95  80]
 [ 98  80   0   0  95  38]
 [ 98  80   0   0  95  38]
 ...
 [176 412 -19  19 120 168]
 [157 431 -19  19 125 168]
 [155 450  -2  19 130 185]]
(150692, 6)
[2 2 2 ... 2 2 2]
(150692,)


# knn

In [6]:
#資料劃分
x_train, x_test, y_train, y_test = train_test_split(feature, answer, test_size=0.3, random_state=9) #9
#參數區間
param_grid = {'n_neighbors':[1, 2, 3]}
#交叉驗證 
cv = StratifiedShuffleSplit(n_splits=2, test_size=0.3, random_state=12)
grid = GridSearchCV(KNeighborsClassifier(), param_grid, cv=cv, verbose=10, n_jobs=6) #n_jobs為平行運算的數量 6
grid.fit(x_train, y_train)
grid_predictions = grid.predict(x_test)

#儲存
file = open('kknn1.pickle', 'wb')
pickle.dump(grid, file)
file.close()

Fitting 2 folds for each of 3 candidates, totalling 6 fits


[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done   2 out of   6 | elapsed:    4.4s remaining:    8.9s
[Parallel(n_jobs=6)]: Done   3 out of   6 | elapsed:    4.4s remaining:    4.4s
[Parallel(n_jobs=6)]: Done   4 out of   6 | elapsed:    4.4s remaining:    2.2s
[Parallel(n_jobs=6)]: Done   6 out of   6 | elapsed:    4.6s remaining:    0.0s
[Parallel(n_jobs=6)]: Done   6 out of   6 | elapsed:    4.6s finished


In [7]:
#最佳參數
print(grid.best_params_)
#預測結果
#print(grid_predictions)
#混淆矩陣
print(confusion_matrix(y_test, grid_predictions))
#分類結果
print(classification_report(y_test, grid_predictions))

{'n_neighbors': 1}
[[22286    26    46]
 [   45  8337     0]
 [   26     0 14442]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     22358
           1       1.00      0.99      1.00      8382
           2       1.00      1.00      1.00     14468

    accuracy                           1.00     45208
   macro avg       1.00      1.00      1.00     45208
weighted avg       1.00      1.00      1.00     45208



# random forest

In [8]:
x_train, x_test, y_train, y_test = train_test_split(feature, answer, test_size=0.3, random_state = 7) #0.3 0(9)0會很多delay
#參數區間
forest = RandomForestClassifier(n_estimators=195, max_depth=130) #195 130
forest = forest.fit(x_train, y_train)
predicted = forest.predict(x_test)
file = open("rrandom1.pickle", "wb")
pickle.dump(forest, file)
file.close()
print("Accuracy score (validation): {0:.3f}".format(forest.score(x_test, y_test)))
print("Confusion Matrix for Raandom Forests:")
print(confusion_matrix(y_test, predicted))
print()
print("Classification Report for Random Forests")
print(classification_report(y_test, predicted))

Accuracy score (validation): 0.998
Confusion Matrix for Raandom Forests:
[[22317     3    34]
 [   27  8290     0]
 [   31     0 14506]]

Classification Report for Random Forests
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     22354
           1       1.00      1.00      1.00      8317
           2       1.00      1.00      1.00     14537

    accuracy                           1.00     45208
   macro avg       1.00      1.00      1.00     45208
weighted avg       1.00      1.00      1.00     45208



# 2P

In [9]:
file = open("./log/P1(1).pickle", "rb")
data = pickle.load(file)
file.close()

In [10]:
game_info = data["ml_2P"]["scene_info"]
game_command = data["ml_2P"]["command"]
print(game_info)
print(game_command)

[{'frame': 0, 'status': 'GAME_ALIVE', 'ball': (98, 80), 'ball_speed': (0, 0), 'platform_1P': (80, 420), 'platform_2P': (80, 50), 'blocker': (40, 240)}, {'frame': 1, 'status': 'GAME_ALIVE', 'ball': (98, 80), 'ball_speed': (0, 0), 'platform_1P': (80, 420), 'platform_2P': (80, 50), 'blocker': (35, 240)}, {'frame': 2, 'status': 'GAME_ALIVE', 'ball': (98, 80), 'ball_speed': (0, 0), 'platform_1P': (80, 420), 'platform_2P': (80, 50), 'blocker': (30, 240)}, {'frame': 3, 'status': 'GAME_ALIVE', 'ball': (98, 80), 'ball_speed': (0, 0), 'platform_1P': (80, 420), 'platform_2P': (80, 50), 'blocker': (25, 240)}, {'frame': 4, 'status': 'GAME_ALIVE', 'ball': (98, 80), 'ball_speed': (0, 0), 'platform_1P': (80, 420), 'platform_2P': (80, 50), 'blocker': (20, 240)}, {'frame': 5, 'status': 'GAME_ALIVE', 'ball': (98, 80), 'ball_speed': (0, 0), 'platform_1P': (80, 420), 'platform_2P': (80, 50), 'blocker': (15, 240)}, {'frame': 6, 'status': 'GAME_ALIVE', 'ball': (98, 80), 'ball_speed': (0, 0), 'platform_1P': (

In [11]:
for i in range(38, 88): #71開始18 17個 2-88
    path = "./log/P1(" + str(i) + ").pickle"
    file = open(path, "rb")
    data = pickle.load(file)
    game_info = game_info + data['ml_2P']['scene_info']
    game_command = game_command + data['ml_2P']['command']
    file.close()  
for i in range(45, 95): #78開始18 17個  9-95
    path = "./log/P2(" + str(i) + ").pickle"
    file = open(path, "rb")
    data = pickle.load(file)
    game_info = game_info + data['ml_2P']['scene_info']
    game_command = game_command + data['ml_2P']['command']
    file.close()
print(len(game_info))
print(len(game_command))

150694
150694


In [12]:
g = game_info[1]

feature = np.array([g["ball"][0], g["ball"][1], g["ball_speed"][0], g["ball_speed"][1], g["platform_2P"][0]+15,80])

print(feature)
print(game_command[1])
if game_command[1] == "NONE":
	game_command[1] = 0
elif game_command[1] == "MOVE_LEFT":
	game_command[1] = 1
else:
	game_command[1] = 2

[98 80  0  0 95 80]
None


In [13]:
for i in range(2, len(game_info) - 1):
    g = game_info[i]
    g_last = game_info[i-1]
    ball_x=g["ball"][0]
    ball_y=g["ball"][1]
    ball_vx=g['ball'][0] - g_last['ball'][0]
    ball_vy=g['ball'][1] - g_last['ball'][1]
    des_x = 80
    #if des_vy > 0:
    if ball_vy < 0:                                             #ball move up
                    if ball_vx < 0:                                         #ball move left
                        des_x = ball_x - (ball_y - 80)
                    else:                                                       #ball move right   
                        des_x = ball_x + (ball_y - 80)
    else:                                                           #ball move down  
                    if ball_vx < 0:                                         #ball move left
                        des_x = ball_x - (680 - (ball_y - 80))
                    else:                                                       #ball move right
                        des_x = ball_x + (680 - (ball_y - 80))
       
    while des_x > 200 or des_x < 0:
        if des_x > 200:
                des_x = 200 - (des_x - 200)
        else: 
                des_x = -des_x
    feature = np.vstack((feature, [g['ball'][0], g['ball'][1],ball_vx, ball_vy,g['platform_2P'][0]+15,des_x]))
    if game_command[i] == "NONE": game_command[i] = 0
    elif game_command[i] == "MOVE_LEFT": game_command[i] = 1
    else: game_command[i] = 2 
        
answer = np.array(game_command[1:-1])
print(feature)
print(feature.shape)
print(answer)
print(answer.shape)

[[ 98  80   0   0  95  80]
 [ 98  80   0   0  95  22]
 [ 98  80   0   0  95  22]
 ...
 [176 412 -19  19 130 172]
 [157 431 -19  19 135 172]
 [155 450  -2  19 140 155]]
(150692, 6)
[2 2 2 ... 2 2 2]
(150692,)


# knn

In [14]:
#資料劃分
x_train, x_test, y_train, y_test = train_test_split(feature, answer, test_size=0.3, random_state=9) #9
#參數區間
param_grid = {'n_neighbors':[1, 2, 3]}
#交叉驗證 
cv = StratifiedShuffleSplit(n_splits=2, test_size=0.3, random_state=12)
grid = GridSearchCV(KNeighborsClassifier(), param_grid, cv=cv, verbose=10, n_jobs=6) #n_jobs為平行運算的數量 #6
grid.fit(x_train, y_train)
grid_predictions = grid.predict(x_test)
#儲存
file = open('kknn2.pickle', 'wb')
pickle.dump(grid, file)
file.close()

#最佳參數
print(grid.best_params_)
#預測結果
#print(grid_predictions)
#混淆矩陣
print(confusion_matrix(y_test, grid_predictions))
#分類結果
print(classification_report(y_test, grid_predictions))

Fitting 2 folds for each of 3 candidates, totalling 6 fits


[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done   2 out of   6 | elapsed:    3.0s remaining:    6.0s
[Parallel(n_jobs=6)]: Done   3 out of   6 | elapsed:    3.0s remaining:    3.0s
[Parallel(n_jobs=6)]: Done   4 out of   6 | elapsed:    3.0s remaining:    1.5s
[Parallel(n_jobs=6)]: Done   6 out of   6 | elapsed:    3.1s remaining:    0.0s
[Parallel(n_jobs=6)]: Done   6 out of   6 | elapsed:    3.1s finished


{'n_neighbors': 1}
[[22353    17    44]
 [   54  8671     0]
 [   28     0 14041]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     22414
           1       1.00      0.99      1.00      8725
           2       1.00      1.00      1.00     14069

    accuracy                           1.00     45208
   macro avg       1.00      1.00      1.00     45208
weighted avg       1.00      1.00      1.00     45208



# random forest

In [15]:
x_train, x_test, y_train, y_test = train_test_split(feature, answer, test_size=0.3, random_state = 7) #0.3 9(0)0會很多delay
#參數區間

forest = RandomForestClassifier(n_estimators=195, max_depth=130) #195 130
forest = forest.fit(x_train, y_train)
predicted = forest.predict(x_test)
file = open("rrandom2.pickle", "wb")
pickle.dump(forest, file)
file.close()
print("Accuracy score (validation): {0:.3f}".format(forest.score(x_test, y_test)))
print("Confusion Matrix for Raandom Forests:")
print(confusion_matrix(y_test, predicted))
print()
print("Classification Report for Random Forests")
print(classification_report(y_test, predicted))

Accuracy score (validation): 0.997
Confusion Matrix for Raandom Forests:
[[22458    10    42]
 [   40  8426     0]
 [   24     0 14208]]

Classification Report for Random Forests
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     22510
           1       1.00      1.00      1.00      8466
           2       1.00      1.00      1.00     14232

    accuracy                           1.00     45208
   macro avg       1.00      1.00      1.00     45208
weighted avg       1.00      1.00      1.00     45208

